In [1]:
import os
from os.path import join as p_join
import sys
sys.path.insert(0, '..')
import time
from tqdm.notebook import tqdm
import itertools
import collections
from datetime import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
import scipy
from matplotlib_venn import *
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
pd.set_option('max.columns', 100)

In [4]:
DATE = '2020-01-01'
PROJECT_PATH = '..'

In [5]:
from src.parse_utils import (
    parse_supplies,
    parse_purchases,
    parse_sales,
    parse_storage,
    parse_report
)
from src.utils import create_wb_db_connection
from src.aggregate import make_daily_sales

Инициализируем подключение к db...
ок


In [6]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

---

In [27]:
wb_key = open(p_join(PROJECT_PATH, 'configs', 'wildberries_api64.txt'), mode='r', encoding='utf-8').read()
var_name_dict = json.load(
    open(p_join(PROJECT_PATH, 'configs', 'wb_columns_dict.json'), mode='r', encoding='utf-8')
)
eng = create_wb_db_connection()

In [8]:
DATE = '2021-01-01'

In [31]:
supplies_df['lastChangeDate'].value_counts()

2021-05-21 05:20:11.100    22
2021-10-22 00:40:25.150    16
2021-05-04 02:00:30.930    13
2021-03-24 11:00:22.460    11
2021-09-06 09:30:00.683    10
2021-09-06 09:20:13.677     9
2021-12-03 19:01:41.600     6
2021-06-30 05:20:20.703     5
2021-08-01 12:40:06.500     5
2021-11-03 04:40:25.803     5
2021-07-11 01:40:10.217     4
2021-06-27 01:04:33.597     3
2021-12-15 20:20:26.170     3
2021-06-30 05:23:57.850     1
Name: lastChangeDate, dtype: int64

In [29]:
# Поставки
supplies_df = parse_supplies(
    date_from=DATE,
    wb_key=wb_key,
#     var_name_dict=var_name_dict
)
supplies_df

,incomeId,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,dateClose,warehouseName,nmId,status
0,2267245,,2021-03-22,2021-03-24 11:00:22.460,КН/001_розовый,0,2001254432003,11,0,2021-03-24,Подольск2,19463069,Принято
1,2267245,,2021-03-22,2021-03-24 11:00:22.460,КН/002_мятный,0,2001254428006,18,0,2021-03-24,Подольск2,19463070,Принято
2,2267245,,2021-03-22,2021-03-24 11:00:22.460,КН/003_серый,0,2001254441005,15,0,2021-03-24,Подольск2,19463071,Принято
3,2267245,,2021-03-22,2021-03-24 11:00:22.460,НСП/004_золотой_глянцевый,0,2001256726001,25,0,2021-03-24,Подольск2,19464860,Принято
4,2267245,,2021-03-22,2021-03-24 11:00:22.460,НСП/005_розовое_золото_глянцевый,0,2001256873002,10,0,2021-03-24,Подольск2,19464861,Принято
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,4804244,,2021-11-30,2021-12-03 19:01:41.600,НСП/010_черный_матовый,0,2001256914002,2,0,2021-12-03,Подольск2,19464866,Принято
109,4804244,,2021-11-30,2021-12-03 19:01:41.600,НСП/011_серебряный_матовый,0,2001256932006,3,0,2021-12-03,Подольск2,19464867,Принято
110,5560043,,2021-12-15,2021-12-15 20:20:26.170,НСП/006_черный_глянцевый,0,2001256882004,1,0,2021-12-15,Подольск2,19464862,Принято
111,5560043,,2021-12-15,2021-12-15 20:20:26.170,mskmsk,0,2002387186009,49,0,2021-12-15,Подольск2,26999508,Принято


In [31]:
date_from = '2021-01-01'
storage_df = requests.get(
    url=f" https://suppliers-stats.wildberries.ru/api/v1/supplier/stocks?dateFrom={date_from}&key={wb_key}"
)
storage_df = pd.DataFrame(storage_df.json())

In [34]:
storage_df

,lastChangeDate,supplierArticle,techSize,barcode,quantity,isSupply,isRealization,quantityFull,quantityNotInOrders,warehouseName,inWayToClient,inWayFromClient,nmId,subject,category,daysOnSite,brand,SCCode,Price,Discount
0,2022-01-09T02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,6,True,False,10,6,Коледино,4,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
1,2022-01-09T02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,2,True,False,2,2,Санкт-Петербург,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
2,2022-01-09T02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,43,True,False,43,38,Электросталь,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
3,2022-01-09T02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,2,True,False,2,2,Краснодар 2,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
4,2022-01-09T02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,0,True,False,1,1,Коледино,1,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
5,2022-01-09T02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,1,True,False,1,1,Санкт-Петербург,0,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
6,2022-01-09T02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,0,True,False,2,2,МЛП-Подольск,0,2,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
7,2022-01-09T02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,1,True,False,1,1,Казань,0,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
8,2022-01-09T02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,12,True,False,12,12,Электросталь,0,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
9,2022-01-09T02:24:46.007,НСП/006_черный_глянцевый,0,2001256882004,66,True,False,76,68,Коледино,10,0,19464862,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50


In [23]:
# Склад
storage_df = parse_storage(
    date_from=DATE,
    wb_key=wb_key,
#     var_name_dict=var_name_dict
)
storage_df

,lastChangeDate,supplierArticle,techSize,barcode,quantity,isSupply,isRealization,quantityFull,quantityNotInOrders,warehouseName,inWayToClient,inWayFromClient,nmId,subject,category,daysOnSite,brand,SCCode,Price,Discount
0,2022-01-09 02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,6,True,False,10,6,Коледино,4,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
1,2022-01-09 02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,2,True,False,2,2,Санкт-Петербург,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
2,2022-01-09 02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,43,True,False,43,38,Электросталь,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
3,2022-01-09 02:24:46.007,НСП/004_золотой_глянцевый,0,2001256726001,2,True,False,2,2,Краснодар 2,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
4,2022-01-09 02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,0,True,False,1,1,Коледино,1,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
5,2022-01-09 02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,1,True,False,1,1,Санкт-Петербург,0,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
6,2022-01-09 02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,0,True,False,2,2,МЛП-Подольск,0,2,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
7,2022-01-09 02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,1,True,False,1,1,Казань,0,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
8,2022-01-09 02:24:46.007,НСП/005_розовое_золото_глянцевый,0,2001256873002,12,True,False,12,12,Электросталь,0,0,19464861,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50
9,2022-01-09 02:24:46.007,НСП/006_черный_глянцевый,0,2001256882004,66,True,False,76,68,Коледино,10,0,19464862,Наборы столовых приборов,Посуда и инвентарь,324,Auplum,,4240,50


In [24]:
storage_df.lastChangeDate.value_counts().sort_index(ascending=False)

2022-01-09 02:24:46.007    44
Name: lastChangeDate, dtype: int64

In [16]:
# Заказы
purchase_df = parse_purchases(
    date_from=DATE,
    wb_key=wb_key,
    flag=0,
#     var_name_dict=var_name_dict
)
purchase_df

,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,warehouseName,oblast,incomeID,odid,nmId,subject,category,brand,isCancel,cancel_dt,gNumber
0,41550604895,2021-12-23 23:21:38,2021-12-24 00:44:07,mskmsk,0,2002387186009,1,780.00,50,Коледино,Московская,3964378,150273366984,26999508,Маски для сна,Аксессуары,syho,False,0001-01-01T00:00:00,1393074529590626661
1,41462752895,2021-12-23 10:54:55,2021-12-24 01:08:14,mskmsk,0,2002387186009,1,780.00,50,Коледино,Ставропольский,3964378,200314251672,26999508,Маски для сна,Аксессуары,syho,False,0001-01-01T00:00:00,97874944964895057463
2,41518553663,2021-12-23 18:59:59,2021-12-24 01:12:13,mskmsk,0,2002387186009,1,780.00,50,Коледино,Тульская,3964378,150268388100,26999508,Маски для сна,Аксессуары,syho,False,0001-01-01T00:00:00,95331710554162071748
3,41505312127,2021-12-23 16:57:46,2021-12-24 03:04:02,petklt01black,0,2006959886007,1,998.26,50,Коледино,Алтай,3964378,150266114184,38129104,Колтунорезки,Товары для животных,imilive,False,0001-01-01T00:00:00,6938731584830987887
4,41529340223,2021-12-23 20:30:20,2021-12-24 03:04:02,НСП/006_черный_глянцевый,0,2001256882004,1,3180.00,57,Коледино,Свердловская,3964378,150270091706,19464862,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,94050797557580456043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,42741511231,2022-01-08 11:57:26,2022-01-08 17:06:13,bandage1,XL,2002852035009,1,1595.74,53,Екатеринбург,Челябинская,3964378,150760237167,28678954,Бандажи коленные,Здоровье,syho,False,0001-01-01T00:00:00,105496199162176957
143,42658822751,2022-01-07 09:30:59,2022-01-08 17:06:13,НСП/004_золотой_глянцевый,0,2001256726001,1,2798.00,50,Екатеринбург,Свердловская,4804109,200808029237,19464860,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,3900328627304207872
144,42658822751,2022-01-07 09:30:59,2022-01-08 17:06:13,НСП/004_золотой_глянцевый,0,2001256726001,1,2798.00,50,Екатеринбург,Свердловская,4804109,200808029238,19464860,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,3900328627304207872
145,42658822751,2022-01-07 09:30:59,2022-01-08 17:06:13,НСП/004_золотой_глянцевый,0,2001256726001,1,2798.00,50,Екатеринбург,Свердловская,4804109,200808029239,19464860,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,3900328627304207872


In [9]:
# Продажи
sales_df = parse_sales(
    date_from=DATE,
    wb_key=wb_key,
    flag=0,
#     var_name_dict=var_name_dict
)
sales_df

,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,isSupply,isRealization,orderId,promoCodeDiscount,warehouseName,countryName,oblastOkrugName,regionName,incomeID,saleID,odid,spp,forPay,finishedPrice,priceWithDisc,nmId,subject,category,brand,IsStorno,gNumber
0,,2021-09-01 10:19:45,2021-09-01 10:30:51,НСП/004_золотой_глянцевый,0,2001256726001,1,2900,30,False,True,3.054720e+10,25,Новосибирск,Россия,Сибирский федеральный округ,Новосибирская,3537166,S1741183025,100649433498,0,1385.47,1386.47,1522.50,19464860,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,91042353372731518546
1,,2021-09-01 14:06:51,2021-09-01 14:14:54,КН/002_мятный,0,2001254428006,1,4104,30,False,True,3.055047e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,2267245,S1742126327,100649515477,0,1960.69,1961.69,2154.60,19463070,Наборы кухонных принадлежностей,Посуда и инвентарь,Auplum,0,9859120058773614172
2,1272674075,2021-09-01 15:45:31,2021-09-02 02:23:36,КН/002_мятный,0,2001254428006,-1,-4104,30,False,True,3.055047e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,2267245,R68974662,100649515477,0,-1960.69,-1961.69,-2154.60,19463070,Наборы кухонных принадлежностей,Посуда и инвентарь,Auplum,0,9859120058773614172
3,,2021-09-02 11:29:10,2021-09-02 11:33:36,НСП/011_серебряный_матовый,0,2001256932006,1,2695,30,False,True,3.069453e+10,25,Екатеринбург,Россия,Уральский федеральный округ,Тюменская,3537166,S1744466980,100653555980,0,1287.54,1288.54,1414.88,19464867,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,91195127552123899025
4,,2021-09-02 14:44:08,2021-09-02 14:49:39,mskmsk,0,2002387186009,1,1332,30,False,True,3.065170e+10,25,Санкт-Петербург,Россия,Северо-западный федеральный округ,Санкт-Петербург,3247649,S1745288671,100652352563,0,594.40,605.00,699.30,26999508,Маски для сна,Аксессуары,syho,0,96786532666405045542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,,2022-01-17 12:45:49,2022-01-17 12:50:11,НСП/009_розовое_золото_матовый,0,2001256903006,1,3973,58,False,True,4.340148e+10,25,Коледино,Россия,Крымский федеральный округ,Симферополь,4804244,S2260761329,201048267963,0,1101.32,1163.00,1251.50,19464865,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,7763568979123178107
1785,16953927276,2022-01-17 00:00:00,2022-01-17 13:20:56,bandage1,,2002852068014,0,0,0,False,True,NaN,0,Подольск,Россия,Центральный федеральный округ,Московская,3964378,D16953927276,200476393734,0,0.00,0.00,0.00,28678954,Бандажи коленные,Здоровье,syho,0,7517348191030456791
1786,,2022-01-17 17:24:54,2022-01-17 17:30:17,mskmsk,0,2002387186009,1,1040,53,False,True,4.331305e+10,25,МЛП-Подольск,Россия,Приволжский федеральный округ,Бирский,5560043,S2262633386,201020484814,0,311.61,322.00,366.60,26999508,Маски для сна,Аксессуары,syho,0,97716158614084256749
1787,,2022-01-17 18:07:56,2022-01-17 18:12:12,mskmsk,0,2002387186009,1,1040,53,False,True,4.335232e+10,25,Санкт-Петербург,Россия,Северо-западный федеральный округ,Мурманская,3964378,S2262964921,150939990686,0,311.61,337.00,366.60,26999508,Маски для сна,Аксессуары,syho,0,3567246212690058117


In [18]:
# Отчёт о продажах по реализации
report_df = parse_report(
    date_from=DATE,
    wb_key=wb_key,
    limit=100_000,
#     var_name_dict=var_name_dict
)
report_df

,realizationreport_id,suppliercontract_code,rrd_id,gi_id,subject_name,nm_id,brand_name,sa_name,ts_name,barcode,doc_type_name,quantity,retail_price,retail_amount,sale_percent,commission_percent,office_name,supplier_oper_name,order_dt,sale_dt,rr_dt,shk_id,retail_price_withdisc_rub,delivery_amount,return_amount,delivery_rub,gi_box_type_name,product_discount_for_report,supplier_promo,rid,ppvz_spp_prc,ppvz_kvw_prc_base,ppvz_kvw_prc,ppvz_sales_commission,ppvz_for_pay,ppvz_reward,ppvz_vw,ppvz_vw_nds,ppvz_office_id,ppvz_office_name,ppvz_supplier_id,ppvz_supplier_name,ppvz_inn,declaration_number,bonus_type_name
0,4646775,None,4355417958,3964378,Миски для животных,38122967,imilive,petmsk01black,0,2006953679001,Продажа,1,3174,1106.93,62.5,0.12,Коледино,Продажа,2021-09-29T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648548264,1190.25,0,0,0,Без коробов,50,25,50721791565,0.0583,0.1000,0.0417,49.59,1047.42,0.0,49.59,9.92,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
1,4646775,None,4355417961,3964378,Наборы столовых приборов,19464864,Auplum,НСП/008_золотой_матовый,0,2001256898005,Продажа,1,2638,1301.85,47.5,0.07,Коледино,Продажа,2021-09-28T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648376766,1384.95,0,0,0,Без коробов,30,25,50720302539,0.0500,0.0583,0.0083,11.54,1288.00,0.0,11.54,2.31,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
2,4646775,None,4355417962,3964378,Бандажи коленные,28678954,syho,bandage1,S,2002852046005,Продажа,0,0,0.00,0.0,0.00,Склад МЛП-Подольск 5,Логистика,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648368725,0.00,0,1,33,Моно,0,0,100719414262,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
3,4646775,None,4355417963,3964378,Органайзеры для хранения,38289400,Auplum,org01gray,0,2007029639004,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648545313,0.00,1,0,105,Моно,0,0,100732857652,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
4,4646775,None,4355417964,3964378,Миски для животных,38122967,imilive,petmsk01black,0,2006953679001,Продажа,1,3174,1130.74,62.5,0.12,Коледино,Продажа,2021-09-29T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648548266,1190.25,0,0,0,Без коробов,50,25,100724134006,0.0417,0.1000,0.0583,69.43,1047.42,0.0,69.43,13.89,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,6567413,None,5254372911,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,1,1040,390.00,62.5,0.15,Коледино,Продажа,2021-12-29T00:00:00Z,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,4597243779,390.00,0,0,0,Без коробов,50,25,200499123973,0.0000,0.1250,0.1250,48.75,331.50,0.0,48.75,9.75,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN
3663,6567413,None,5254372912,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,4597243811,0.00,1,0,55,Микс,0,0,200591735389,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN
3664,6567413,None,5254372913,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,4597243803,0.00,1,0,55,Микс,0,0,200613915065,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN
3665,6567413,None,5254372914,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,4597243807,0.00,1,0,55,Микс,0,0,200593435009,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN


In [53]:
(
    set(supplies_df.columns) 
    & set(storage_df.columns)
    & set(purchase_df.columns)
    & set(sales_df.columns)
#     & set(report_df.columns)
)

{'barcode',
 'lastChangeDate',
 'nmId',
 'quantity',
 'supplierArticle',
 'techSize',
 'warehouseName'}

In [16]:
import sqlalchemy as sa

In [17]:
sql_db_credentials = json.load(
    open(p_join(PROJECT_PATH, 'configs', 'sql_db_creadentials.json'), mode='r', encoding='utf-8')
)

In [20]:
db = 'yarik'
eng = sa.create_engine(
    f"mysql+pymysql://{sql_db_credentials[db]['login']}:{sql_db_credentials[db]['password']}@{sql_db_credentials[db]['host']}:{sql_db_credentials[db]['port']}"
)
eng.connect()

In [21]:
pd.read_sql(
    """
    show databases
    """,
    eng
)

,Database
0,information_schema
1,mp_db
2,mysql
3,performance_schema
4,sys
5,wb_yarik


In [23]:
sales_df.to_sql(
    schema='wb_yarik',
    name='sales',
    con=eng,
    if_exists='replace'
)

In [24]:
purchase_df.to_sql(
    schema='wb_yarik',
    name='purchases',
    con=eng,
    if_exists='replace'
)

In [25]:
storage_df.to_sql(
    schema='wb_yarik',
    name='storage',
    con=eng,
    if_exists='replace'
)

In [26]:
supplies_df.to_sql(
    schema='wb_yarik',
    name='supplies',
    con=eng,
    if_exists='replace'
)

In [27]:
report_df.to_sql(
    schema='wb_yarik',
    name='report',
    con=eng,
    if_exists='replace'
)

In [62]:
daily_sales_df = pd.read_sql(
    """
    with t_agg as 
    (
        select 
            supplierArticle
            , techSize
            , subject
            , category
            , brand
            , cast(date as date) as day
            , sum(quantity) as sum_quantity
            , sum(quantity * totalPrice) as sum_totalPrice
            , sum(quantity * pricewithdisc) as sum_pricewithdisc
            , sum(quantity * forPay) as sum_forPay
            , sum(quantity * finishedPrice) as sum_finishedPrice
        from
            wb_yarik.sales
        where
            quantity > 0
        group by
            supplierArticle
            , techSize
            , subject
            , category
            , brand
            , cast(date as date)
        order by
            cast(date as date) asc
            , subject
            , techSize
            , date asc
    )
    
    select
        *
    from
        t_agg
    """,
    eng
)
daily_sales_df.to_sql(
    schema='wb_yarik',
    name='daily_sales',
    con=eng,
    if_exists='replace'
)

In [17]:
daily_sales_df = pd.read_sql(
    """
    with t_daily_sales as 
        (   
            select 
                supplierArticle
                , techSize
                , subject
                , category
                , brand
                , barcode
                , cast(date as date) as day
                , sum(quantity) as sum_quantity
                , sum(quantity * totalPrice) as sum_totalPrice
                , sum(quantity * pricewithdisc) as sum_pricewithdisc
                , sum(quantity * forPay) as sum_forPay
                , sum(quantity * finishedPrice) as sum_finishedPrice
            from
                wb_yarik.sales
            where
                quantity > 0
            group by
                supplierArticle
                , techSize
                , subject
                , barcode
                , category
                , brand
                , cast(date as date)
            order by
                cast(date as date) asc
                , subject
                , techSize
                , date asc
        )
        
    select * from t_daily_sales
    """,
    eng
)
daily_sales_df.head()

,supplierArticle,techSize,subject,category,brand,barcode,day,sum_quantity,sum_totalPrice,sum_pricewithdisc,sum_forPay,sum_finishedPrice
0,bandage1,XL,Бандажи коленные,Здоровье,syho,2002852035009,2021-12-23,1.0,2264.0,849.00,721.65,849.0
1,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-23,1.0,5357.0,1647.28,1400.19,1647.0
2,НСП/010_черный_матовый,0,Наборы столовых приборов,Посуда и инвентарь,Auplum,2001256914002,2021-12-24,1.0,3973.0,1281.29,1127.54,1281.0
3,НСП/004_золотой_глянцевый,0,Наборы столовых приборов,Посуда и инвентарь,Auplum,2001256726001,2021-12-24,1.0,4240.0,1272.00,1119.36,1272.0
4,org01pink,0,Органайзеры для хранения,Хранение вещей,Auplum,2007029886002,2021-12-24,2.0,5068.0,1900.50,1672.44,1900.0


In [18]:
daily_storage_df = pd.read_sql(
    """
    with t_daily_storage as 
        (   
            select 
                supplierArticle
                , techSize
                , barcode
                , cast(lastChangeDate as date) as day
                , sum(quantityFull) as sum_quantityFull
                , sum(inWayToClient) as sum_inWayToClient
                , sum(inWayFromClient) as sum_inWayFromClient
            from
                wb_yarik.storage
            group by
                supplierArticle
                , techSize
                , barcode
                , cast(lastChangeDate as date)
        )
        
    select * from t_daily_storage
    """,
    eng
)
daily_storage_df.head()

,supplierArticle,techSize,barcode,day,sum_quantityFull,sum_inWayToClient,sum_inWayFromClient
0,НСП/004_золотой_глянцевый,0,2001256726001,2022-01-09,57.0,4.0,0.0
1,НСП/005_розовое_золото_глянцевый,0,2001256873002,2022-01-09,17.0,1.0,2.0
2,НСП/006_черный_глянцевый,0,2001256882004,2022-01-09,76.0,10.0,0.0
3,НСП/007_серебряный_глянцевый,0,2001256892003,2022-01-09,47.0,9.0,2.0
4,НСП/009_розовое_золото_матовый,0,2001256903006,2022-01-09,4.0,1.0,0.0


In [11]:
daily_sales_df = make_daily_sales()
print(f"{daily_sales_df.shape[0]:,} x {daily_sales_df.shape[1]:,}")
(
    daily_sales_df
    .sort_values(by=['subject', 'supplierArticle', 'techSize', 'day'], ascending=True)
)

850 x 22


,supplierArticle,techSize,subject,category,brand,barcode,day,avg_totalPrice,avg_pricewithdisc,avg_forPay,avg_finishedPrice,sum_sales,sum_returned,sum_purchases,sum_q_totalPrice,sum_q_pricewithdisc,sum_q_forPay,sum_q_finishedPrice,sum_q_totalPrice_returned,sum_q_pricewithdisc_returned,sum_q_forPay_returned,sum_q_finishedPrice_returned
823,bandage1,,Бандажи коленные,Здоровье,syho,2002852068014,2022-01-17,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
48,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-12,1505.0,790.13,671.61,781.500000,4.0,0.0,0.0,6020.0,3160.52,2686.44,3126.00,0.0,0.0,0.0,0.0
57,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-13,1505.0,790.13,671.61,789.000000,2.0,0.0,0.0,3010.0,1580.26,1343.22,1578.00,0.0,0.0,0.0,0.0
97,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-18,1505.0,790.13,711.12,719.000000,1.0,0.0,3.0,1505.0,790.13,711.12,719.00,0.0,0.0,0.0,0.0
103,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-19,1505.0,790.13,697.95,723.666667,3.0,0.0,1.0,4515.0,2370.39,2093.85,2171.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-20,5357.0,1647.28,1400.19,1543.000000,2.0,0.0,2.0,10714.0,3294.56,2800.38,3086.00,0.0,0.0,0.0,0.0
741,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-21,5357.0,1647.28,1400.19,1647.000000,1.0,0.0,1.0,5357.0,1647.28,1400.19,1647.00,0.0,0.0,0.0,0.0
748,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-23,5357.0,1647.28,1400.19,1647.000000,1.0,0.0,0.0,5357.0,1647.28,1400.19,1647.00,0.0,0.0,0.0,0.0
752,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-24,5357.0,1828.08,1553.87,1827.940000,2.0,0.0,2.0,10714.0,3656.16,3107.74,3655.88,0.0,0.0,0.0,0.0


In [86]:
sales_df[['supplierArticle', 'subject']].drop_duplicates().sort_values(by=['subject'])

,supplierArticle,subject
25,bandage1,Бандажи коленные
129,petklt01black,Колтунорезки
127,krz01gold,Корзинки
12,teasp_вет/черный,Ложки чайные
18,teasp_вет/серебро,Ложки чайные
49,teasp_вет/золотой,Ложки чайные
52,teasp_вет/розовое_золото,Ложки чайные
4,mskmsk,Маски для сна
27,petmsk01black,Миски для животных
1359,КН/003_серый,Наборы кухонных принадлежностей


In [84]:
sales_df.subject.unique()

array(['Наборы столовых приборов', 'Наборы кухонных принадлежностей',
       'Маски для сна', 'Ложки чайные', 'Бандажи коленные',
       'Миски для животных', 'Органайзеры для хранения', 'Сумки',
       'Корзинки', 'Колтунорезки'], dtype=object)

In [115]:
tmp = pd.read_sql(
    """
    with t_bandage as 
        (
            select  
                day
                , sum(sum_sales) as bandage_s
                -- , sum(sum_purchases) as bandage_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Бандажи коленные'
            group by
                day
            order by
                day asc
        ),

        t_koltunorezki as 
        (
            select  
                day
                , sum(sum_sales) as koltunorezki_s
                -- , sum(sum_purchases) as koltunorezki_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Колтунорезки'
            group by
                day
            order by
                day asc
        ),
        
        t_korzinki as 
        (
            select  
                day
                , sum(sum_sales) as korzinki_s
                -- , sum(sum_purchases) as korzinki_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Корзинки'
            group by
                day
            order by
                day asc
        ),
        
        t_spoons as 
        (
            select  
                day
                , sum(sum_sales) as spoons_s
                -- , sum(sum_purchases) as spoons_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Ложки чайные'
            group by
                day
            order by
                day asc
        ),
        
        t_kitchen as 
        (
            select  
                day
                , sum(sum_sales) as kitchen_s
                -- , sum(sum_purchases) as kitchen_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Наборы кухонных принадлежностей'
            group by
                day
            order by
                day asc
        ),
        
        t_dream_mask as 
        (
            select  
                day
                , sum(sum_sales) as dream_mask_s
                -- , sum(sum_purchases) as dream_mask_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Маски для сна'
            group by
                day
            order by
                day asc
        ),
        
        t_animal_plate as 
        (
            select  
                day
                , sum(sum_sales) as animal_plate_s
                -- , sum(sum_purchases) as animal_plate_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Миски для животных'
            group by
                day
            order by
                day asc
        ),
        
        t_cutlery as 
        (
            select  
                day
                , sum(sum_sales) as cutlery_s
                -- , sum(sum_purchases) as cutlery_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Наборы столовых приборов'
            group by
                day
            order by
                day asc
        ),
        
        t_organizer as 
        (
            select  
                day
                , sum(sum_sales) as organizer_s
                -- , sum(sum_purchases) as organizer_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Органайзеры для хранения'
            group by
                day
            order by
                day asc
        ),
        
        t_bags as 
        (
            select  
                day
                , sum(sum_sales) as bags_s
                -- , sum(sum_purchases) as bags_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Сумки'
            group by
                day
            order by
                day asc
        )


    select
        case when
            t1.day is null then t2.day else t1.day end as day
        , case when
            t1.bandage_s is null then 0 else t1.bandage_s end as bandage_s
        , case when
            t2.koltunorezki_s is null then 0 else t2.koltunorezki_s end as koltunorezki_s
        , case when
            t3.korzinki_s is null then 0 else t3.korzinki_s end as korzinki_s
        , case when
            t4.spoons_s is null then 0 else t4.spoons_s end as spoons_s
        , case when
            t5.kitchen_s is null then 0 else t5.kitchen_s end as kitchen_s
        , case when
            t6.dream_mask_s is null then 0 else t6.dream_mask_s end as dream_mask_s
        , case when
            t7.animal_plate_s is null then 0 else t7.animal_plate_s end as animal_plate_s
        , case when
            t8.cutlery_s is null then 0 else t8.cutlery_s end as cutlery_s
        , case when
            t9.organizer_s is null then 0 else t9.organizer_s end as organizer_s
        , case when
            t10.bags_s is null then 0 else t10.bags_s end as bags_s
        
    from
        t_bandage t1
        left join
            t_koltunorezki t2
        on  
            t1.day = t2.day
            
        left join 
            t_korzinki t3
        on
            t1.day = t3.day
            
        left join 
            t_spoons t4
        on
            t1.day = t4.day
            
        left join 
            t_kitchen t5
        on
            t1.day = t5.day
            
        left join 
            t_dream_mask t6
        on
            t1.day = t6.day
            
        left join 
            t_animal_plate t7
        on
            t1.day = t7.day
            
        left join 
            t_cutlery t8
        on
            t1.day = t8.day
            
        left join 
            t_organizer t9
        on
            t1.day = t9.day
            
        left join 
            t_bags t10
        on
            t1.day = t10.day
    """,
    eng
)
tmp

,day,bandage_s,koltunorezki_s,korzinki_s,spoons_s,kitchen_s,dream_mask_s,animal_plate_s,cutlery_s,organizer_s,bags_s
0,2021-09-13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2021-09-15,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0
2,2021-09-18,1.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0,0.0,0.0
3,2021-09-19,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-09-20,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
5,2021-09-21,1.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,0.0,0.0
6,2021-09-27,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,2.0
7,2021-10-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
8,2021-10-13,2.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,4.0,2.0
9,2021-10-19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0


In [124]:
import plotly.express as px

In [161]:
import plotly.graph_objects as go

plt.figure(figsize=(16, 6))
colors = ['blue', 'red', 'green', 'orange', 'yellow', 'pink', 'purple', 'brown', 'black']
colors = colors + [color for color in list(matplotlib.colors.cnames.keys()) if color not in colors]
fig = go.Figure()
for i, col in enumerate(tmp.columns[1:]):
    fig.add_trace(go.Scatter(x=tmp['day'], y=tmp[col],
                    mode='lines+markers', name=col))
fig.write_html("/Users/antonandreytsev/Desktop/plotly.html")
fig.show();

<Figure size 1152x432 with 0 Axes>